In [1]:
%cd -q ..

import numpy as np
import pandas as pd

from model.helpers import load_params
from model.model import Model
from model.inpatients import InpatientsModel
from model.outpatients import OutpatientsModel
from model.aae import AaEModel
from model.health_status_adjustment import HealthStatusAdjustmentInterpolated

pd.set_option("display.precision", 1)

In [2]:
import logging
logging.basicConfig(
    format="%(asctime)s %(levelname)-8s %(message)s",
    level=logging.INFO,
    datefmt="%Y-%m-%d %H:%M:%S",
)

In [3]:
params = load_params("sample_params.json")
run_params = Model.generate_run_params(params)
hsa = HealthStatusAdjustmentInterpolated(f"data/{params['dataset']}", params["life_expectancy"])

# Helper Methods

Below are some helper methods for turning the results into useful Pandas `DataFrames`.

In [4]:
def run_model(model):
    params = load_params("sample_params.json")

    return {
        k0: pd.DataFrame([
            { **dict(k1), "value": v1 }
            for k1, v1 in v0.items()
        ])
        for k0, v0 in model(params, "data", hsa, run_params).go(0).items()
    }

In [5]:
def default_pod_measure_pivot(agg):
    default_results = (
        agg["default"]
        .groupby(["pod", "measure"], as_index=False)
        .agg({"value": sum})
        .pivot(index=["pod"], columns="measure")
        .fillna(0)
    )
    default_results.loc["total"] = default_results.sum()

    return default_results

In [6]:
def step_counts_simple(agg):
    step_counts = (
        agg["step_counts"]
        .drop(columns=["strategy", "activity_type"])
        .groupby(["change_factor", "measure"], as_index=False)
        .sum()
        .pivot(index="change_factor", columns="measure")
        .loc[agg["step_counts"]["change_factor"].unique()]
    )
    step_counts.loc["total"] = step_counts.sum()

    return step_counts

# Running the models

## Inpatients

Running the principal model run (model run 0) for the Inpatients data

In [7]:
ip_agg = run_model(InpatientsModel)

### Aggregate Results

The available aggregations for inpatients are:

In [8]:
list(ip_agg.keys())

['default',
 'sex+age_group',
 'sex+tretspef',
 'bed_occupancy',
 'theatres_available',
 'step_counts']

We can view the default aggregations, summarising to pod/measure

In [9]:
default_pod_measure_pivot(ip_agg)

value                                 
measure                   admissions attendances   beddays procedures
pod                                                                  
ip_elective_admission         6984.0         0.0   29784.0     3879.0
ip_elective_daycase          54306.0         0.0   54306.0    29497.0
ip_maternity_admission        7090.0         0.0   30643.0     3770.0
ip_non-elective_admission    44340.0         0.0  161931.0     5924.0
op_procedure                     0.0       935.0       0.0        0.0
total                       112720.0       935.0  276664.0    43070.0

### Estimate of effects

In [10]:
step_counts_simple(ip_agg)

value          
measure                    admissions   beddays
change_factor                                  
baseline                      96864.0  308769.0
demographic_adjustment        22331.6   75279.9
health_status_adjustment      -3398.1  -12619.5
expat                          -210.1    -744.8
repat                          2332.8    7987.7
waiting_list_adjustment         165.0     248.9
baseline_adjustment               0.0       0.0
non-demographic_adjustment    11913.7   40762.0
activity_avoidance           -16343.9  -73932.2
efficiencies                   -935.0  -69087.0
total                        112720.0  276664.0

## Outpatients

Running the principal model run (model run 0) for the Outpatients data

In [11]:
op_agg = run_model(OutpatientsModel)

### Aggregate Results

The available aggregations for outpatients are:

In [12]:
list(op_agg.keys())

['default', 'sex+age_group', 'sex+tretspef', 'step_counts']

We can view the default aggregations, summarising to pod/measure

In [13]:
default_pod_measure_pivot(op_agg)

value                 
measure      attendances tele_attendances
pod                                      
op_first          124125            14319
op_follow-up      290067            36004
op_procedure           0            40618
total             414192            90941

### Estimate of effects

In [14]:
step_counts_simple(op_agg)

value                 
measure                  attendances tele_attendances
change_factor                                        
baseline                    440289.0          14655.0
demographic_adjustment       86405.7           3135.5
health_status_adjustment    -11184.5           -377.2
expat                        -1358.3            -45.0
repat                        10280.8            336.4
waiting_list_adjustment       7063.7            233.1
baseline_adjustment              0.0              0.0
activity_avoidance          -42893.4          -1407.9
efficiencies                -74411.0          74411.0
total                       414192.0          90941.0

## Accident and Emergency

Running the principal model run (model run 0) for the A&E data

In [15]:
ae_agg = run_model(AaEModel)

### Aggregate Results

The available aggregations for outpatients are:

In [16]:
list(ae_agg.keys())

['default', 'sex+age_group', 'step_counts']

We can view the default aggregations, summarising to pod/measure

In [17]:
default_pod_measure_pivot(ae_agg)

value         
measure     ambulance  walk-in
pod                           
aae_type-01   36902.0  66615.0
aae_type-02       6.0    939.0
aae_type-03     222.0  15964.0
aae_type-04       0.0    833.0
total         37130.0  84351.0

### Estimate of effects

In [18]:
step_counts_simple(ae_agg)

,value
measure,arrivals
change_factor,
baseline,116368.0
demographic_adjustment,15704.8
health_status_adjustment,-1295.9
expat,-2620.3
repat,2567.9
baseline_adjustment,0.0
activity_avoidance,-9243.5
total,121481.0
